In [2]:
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
import scipy.integrate as integrate

In [3]:
hazard_rates = pd.read_csv('hazard_rates.csv')
hazard_rates.sort_values(by = 'SettlementDate', inplace=True)
hazard_rates.reset_index(inplace=True, drop=True)
hazard_rates = hazard_rates.iloc[:100,:]

discount_factors = pd.read_csv('discount_factor_common_dates.csv')
discount_factors = discount_factors.set_index('Date')
dfc = discount_factors.columns
discount_factors['zero'] = 1
discount_factors = discount_factors[np.append('zero', dfc )]
discount_factors.sort_values(by = 'Date', inplace=True)
discount_factors.reset_index(inplace=True)
discount_factors = discount_factors.iloc[:100,:]

coupons = pd.read_excel('treasury_bond_note_coupons.xlsx')

cds = pd.read_csv('cds_data_processed.csv')
cds.sort_values(by = 'Settlement Date', inplace=True)
cds.reset_index(inplace=True, drop=True)
cds = cds.iloc[:100,:]

In [4]:
mean_coupons = coupons.groupby('Tenor').mean().reset_index()

In [5]:
cds['Open_6m'] = cds['Open_6m']/100
cds['Open_1yr'] = cds['Open_1yr']/100
cds['Open_2yr'] = mean_coupons[mean_coupons['Tenor'] == 2]['Cpn'].values[0] - cds['Open_2yr']/100
cds['Open_3yr'] = mean_coupons[mean_coupons['Tenor'] == 3]['Cpn'].values[0] - cds['Open_3yr']/100
cds['Open_3yr'] = mean_coupons[mean_coupons['Tenor'] == 5]['Cpn'].values[0] - cds['Open_5yr']/100
cds['Open_3yr'] = mean_coupons[mean_coupons['Tenor'] == 7]['Cpn'].values[0] - cds['Open_7yr']/100
cds['Open_3yr'] = mean_coupons[mean_coupons['Tenor'] == 10]['Cpn'].values[0] - cds['Open_10yr']/100

In [6]:
time_periods_m = np.array([0, 1/12,3/12, 6/12, 9/12, 1])
time_periods_y = np.arange(2, 31, 1)
time_periods = np.append(time_periods_m, time_periods_y)

In [7]:
discount_factors_funcs = discount_factors.apply(lambda row: interp1d(time_periods, row[1:], kind='cubic'), axis = 1)

In [8]:
cds_time_periods = [0.5, 1, 2, 3, 5, 7, 10]

In [9]:
discount_factors_subset_times = discount_factors.iloc[:, [4, 6, 7, 8,10,12,15]]
cds_subset_times = cds.iloc[:, 8:]

In [10]:
df_integrate = np.empty(shape =(cds.shape[0],len(cds_time_periods )))
for i in range(discount_factors.shape[0]):
    for j, val in enumerate(cds_time_periods):
        df_integrate[i, j]  = integrate.quad(lambda x: discount_factors_funcs[i](x), 0,val)[0]

In [11]:
hz_moded = hazard_rates.copy()
hz_moded
hz_moded['Maturity_6m'] = np.exp(-hz_moded['Maturity_6m']*0.5)
hz_moded['Maturity_1yr'] = np.exp(-hz_moded['Maturity_1yr']*1)
hz_moded['Maturity_2yr'] = np.exp(-hz_moded['Maturity_1yr']*2)
hz_moded['Maturity_3yr'] = np.exp(-hz_moded['Maturity_1yr']*3)
hz_moded['Maturity_5yr'] = np.exp(-hz_moded['Maturity_1yr']*5)
hz_moded['Maturity_7yr'] = np.exp(-hz_moded['Maturity_1yr']*7)
hz_moded['Maturity_10yr'] = np.exp(-hz_moded['Maturity_1yr']*10)


In [12]:
hz_moded_subset_times = hz_moded.iloc[:, 1:]

In [13]:
pt_2 = np.multiply(np.add(np.multiply(cds_subset_times, df_integrate), discount_factors_subset_times), hz_moded_subset_times)

/var/folders/34/gh568vr946d45y78dns07shm0000gn/T/ipykernel_50556/4021414376.py:1: FutureWarning: Calling a ufunc on non-aligned DataFrames (or DataFrame/Series combination). Currently, the indices are ignored and the result takes the index/columns of the first DataFrame. In the future , the DataFrames/Series will be aligned before applying the ufunc.
Convert one of the arguments to a NumPy array (eg 'ufunc(df1, np.asarray(df2)') to keep the current behaviour, or align manually (eg 'df1, df2 = df1.align(df2)') before passing to the ufunc to obtain the future behaviour and silence this warning.
  pt_2 = np.multiply(np.add(np.multiply(cds_subset_times, df_integrate), discount_factors_subset_times), hz_moded_subset_times)


In [14]:
integrate.quad(lambda t: integrate.quad(lambda x: discount_factors_funcs[i](x), 0,t)[0], 0, val)[0]

46.569316309241266

In [ ]:
df_integrate_double = np.empty(shape =(cds.shape[0],len(cds_time_periods )))
for i in range(discount_factors.shape[0]):
    for j, val in enumerate(cds_time_periods):
        df_integrate_double[i, j]  = integrate.quad(lambda t: hz_moded_subset_times[i,j] * np.exp(-hz_moded_subset_times[i,j]*t) * integrate.quad(lambda x: discount_factors_funcs[i](x), 0,t)[0] + discount_factors_funcs[i](t), 0, val)[0]

In [18]:
df_integrate_double = np.empty(shape =(cds.shape[0],len(cds_time_periods )))
for i in range(discount_factors.shape[0]):
    for j, val in enumerate(cds_time_periods):
        df_integrate_double[i, j] = integrate.quad(lambda t: hz_moded_subset_times.iloc[i,j] * np.exp(-hz_moded_subset_times.iloc[i,j]*t) * cds_subset_times.iloc[i,j]* integrate.quad(lambda x: discount_factors_funcs[i](x), 0,t)[0] + discount_factors_funcs[i](t), 0, val)[0]

In [21]:
pt_2 + df_integrate_double

,Open_6m,Open_1yr,Open_2yr,Open_3yr,Open_5yr,Open_7yr,Open_10yr
0,1.646500,2.310415,3.365705,4.906065,9.384170,7.575275,8.750215
1,1.657918,2.335111,3.354415,4.904108,9.738937,7.662688,8.819084
2,1.641010,2.298713,3.373442,4.920027,9.231869,7.552071,8.814011
3,1.637763,2.291872,3.370707,4.910436,9.025242,7.486934,8.817402
4,1.626031,2.266451,3.386512,4.925152,8.739844,7.422736,8.773634
...,...,...,...,...,...,...,...
95,1.582735,2.173413,3.437068,4.963909,7.410427,7.167069,8.801854
96,1.583330,2.175329,3.435180,4.958185,7.416269,7.183995,8.811061
97,1.583394,2.175766,3.435657,4.953375,7.422010,7.222706,8.832766
98,1.583685,2.176660,3.433338,4.947440,7.412951,7.229383,8.842069
